In [1]:
from sagas.nlu.ruleset_procs import list_words, cached_chunks, get_main_domains
from sagas.conf.conf import cf
from sagas.tracker_jupyter import enable_jupyter_tracker
enable_jupyter_tracker()

# She denied being my mother
sents='Ela negou ser minha mãe.'
lang='pt'
domain='verb_domains'
chunks = cached_chunks(sents, lang, cf.engine(lang))
domains = chunks[domain]
domains

[{'index': 2,
  'text': 'negou',
  'lemma': 'negar',
  'upos': 'VERB',
  'xpos': '_',
  'feats': 'Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin',
  'governor': 0,
  'dependency_relation': 'root',
  'entity': [],
  'segments': [],
  'nsubj': [{'index': 1,
    'text': 'Ela',
    'lemma': 'ela',
    'upos': 'PRON',
    'xpos': '_',
    'feats': 'Case=Nom|Gender=Fem|Number=Sing|Person=3|PronType=Prs',
    'governor': 2,
    'dependency_relation': 'nsubj',
    'entity': [],
    'segments': []}],
  'punct': [{'index': 6,
    'text': '.',
    'lemma': '.',
    'upos': 'PUNCT',
    'xpos': '_',
    'feats': '_',
    'governor': 2,
    'dependency_relation': 'punct',
    'entity': [],
    'segments': []}],
  'obl': [{'index': 5,
    'text': 'mãe',
    'lemma': 'mãe',
    'upos': 'NOUN',
    'xpos': '_',
    'feats': 'Gender=Fem|Number=Sing',
    'governor': 2,
    'dependency_relation': 'obl',
    'entity': [],
    'segments': [],
    'cop': [{'index': 3,
      'text': 'ser',
      'lem

In [2]:
[w.text for w in chunks['doc'].words]

['Ela', 'negou', 'ser', 'minha', 'mãe', '.']

In [3]:
from sagas.nlu.uni_chunks import get_chunk
import sagas

cla='obl'
result=get_chunk(chunks, domain, cla, lambda w: sagas.to_obj({'rel':w.dependency_relation, 
                                                 'pos':w.upos.lower(), 
                                                 'word':f"{w.text}/{w.lemma}"}))
result

[{'rel': 'cop', 'pos': 'aux', 'word': 'ser/ser'},
 {'rel': 'det', 'pos': 'det', 'word': 'minha/meu'},
 {'rel': 'obl', 'pos': 'noun', 'word': 'mãe/mãe'}]

In [4]:
from sagas.nlu.nlu_cli import retrieve_word_info
rel='cop'
word=next((w.word for w in result if w.rel==rel), None)
if word:
    print(word)
    rs = retrieve_word_info('get_synsets', word, lang, '*')
    print(rs)

ser/ser
['entity.n.01', 'organism.n.01', 'animal.n.01', 'debut.v.03', 'exist.v.01', 'be.v.01', 'creature.n.02', 'being.n.01']


In [5]:
def check_chain(kind, word, pos, lang):
    from sagas.nlu.synonyms import synonyms
    from sagas.nlu.utils import predicate
    
    r=synonyms.match(word, lang)
    if r is None:
        return predicate(kind, word, lang, pos)
    return predicate(kind, r, 'en', pos)

check_chain('be', word, '*', lang)

True

In [6]:
cats={'be'}
any([check_chain(cat, word, '*', lang) for cat in cats])

True

In [7]:
from sagas.nlu.inspector_clauses import check_clause_sub
check_clause_sub(sents, 'pt', 'verb_domains', 'obl', 'cop', {'be'})

True

In [3]:
from sagas.nlu.inspector_common import cla_meta
from sagas.nlu.inspector_clauses import ClausesInspector as clauses, cla_expr

e=cla_expr('verb:obl', cop={'be'})
e.run('', cla_meta('Ela negou ser minha mãe.', 'pt'))

True